###Data Loading

In [0]:
#python list to dataframe

data = [
    ("Alice", 1, "New York"),
    ("Bob", 2, "London"),
    ("Charlie", 3, "Paris")
]

df=spark.createDataFrame(data,["Name", "ID", "City"])
df.show()

In [0]:
#loading csv data

df=spark.read.csv("/FileStore/tables/BigMart_Sales.csv", header=True, inferSchema=True)

In [0]:
# df.printSchema()
# df.show()
df.limit(5).display()

##schema

In [0]:
#ddl schema
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

my_ddl_schema= """
                    Item_Identifier STRING,
                    Item_Weight DOUBLE,
                    Item_Fat_Content STRING,
                    Item_Visibility DOUBLE,
                    Item_Type STRING,
                    Item_MRP DOUBLE,
                    Outlet_Identifier STRING,
                    Outlet_Establishment_Year INT,
                    Outlet_Size STRING,
                    Outlet_Location_Type STRING,
                    Outlet_Type STRING,
                    Item_Outlet_Sales DOUBLE
                """

#schema defined using structType
my_struct_schema=StructType([
    StructField("Item_Identifier", StringType(), True),
    StructField("Item_Weight", DoubleType(), True),
    StructField("Item_Fat_Content", StringType(), True),
    StructField("Item_Visibility", DoubleType(), True),
    StructField("Item_Type", StringType(), True),
    StructField("Item_MRP", DoubleType(), True),
    StructField("Outlet_Identifier", StringType(), True),
    StructField("Outlet_Establishment_Year", IntegerType(), True),
    StructField("Outlet_Size", StringType(), True),
    StructField("Outlet_Location_Type", StringType(), True),
    StructField("Outlet_Type", StringType(), True),
    StructField("Item_Outlet_Sales", DoubleType(), True)
])


In [0]:
df=spark.read.csv("/FileStore/tables/BigMart_Sales.csv", header=True, schema=my_struct_schema)
df.printSchema()
df.display()

### Column Operations

In [0]:
from pyspark.sql.functions import col
from pyspark.sql import functions as F

#Column Operations

#select
df.select("Item_Identifier").limit(5).display()

#alias
df.select(col("Item_Identifier").alias("Item_ID"),col("Item_Weight"),col("Item_Fat_Content")).limit(5).display()

#withColumn
df_with_temp = df.withColumn("temp", F.col("Item_Weight") + 2)

#withColumnRenamed
df_renamed = df_with_temp.withColumnRenamed("temp", "temp_column")
display(df_renamed.limit(5))

#drop column
df_renamed.drop("temp_column").limit(5).display()

#type casting
df.withColumn("Item_Weight",col("Item_Weight").cast(StringType())).limit(5).display()

#replacing values
df.withColumn("Item_Visibility",F.round(F.col("Item_Visibility"),2)).limit(5).display()

### Row Operations

In [0]:
# row operations

#Filter
df.filter(col("Item_Fat_Content") == "Regular").limit(5).display()
df.filter((col("Item_Type") == "Soft Drinks") & (col("Item_Weight") > 5)).limit(5).display()
df.filter(
    (col("Outlet_Location_Type").isin("Tier 1", "Tier 2"))
    & (col("Outlet_Size").isNull())
).limit(5).display()

#distinct
df.distinct().limit(5).display()

#sort
df.sort(col("Item_Weight").desc()).limit(5).display()
df.sort([col("Item_Weight"),col("Item_MRP")],ascending=[0,0]).limit(5).display()

#drop duplicates
df.dropDuplicates().display()
df.dropDuplicates(subset=["Item_Type"]).display()

### Unions

In [0]:
#Union

df1_data = [("Apple", 100), ("Banana", 150)]
df1_schema = StructType([
    StructField("item", StringType(), True),
    StructField("quantity", IntegerType(), True)
])
df1 = spark.createDataFrame(df1_data, schema=df1_schema)

# df1.display()

df2_data = [("Orange", 200), ("Apple", 120)]
df2_schema = StructType([
    StructField("item", StringType(), True),
    StructField("quantity", IntegerType(), True)
])
df2 = spark.createDataFrame(df2_data, schema=df2_schema)

df3_data = [(50, "Grape")]
df3_schema = StructType([
    StructField("quantity", IntegerType(), True),
    StructField("item", StringType(), True) # Order is different from df1 and df2
])
df3 = spark.createDataFrame(df3_data, schema=df3_schema)

# df2.display()

df1.union(df2).display()
df1.unionByName(df2).unionByName(df3).display()

### Date Functions

In [0]:
from pyspark.sql.functions import current_date, date_add, date_sub, datediff, date_format

#Date Functions

#current date
df.withColumn("current_date",current_date()).limit(1).display()

#date add
df.withColumn("week_after",date_add("current_date",7)).limit(1).display()

#date sub
df_with_week_before=df.withColumn("week_before",date_sub("current_date",7))
# df_with_week_before=df.withColumn("week_after",date_add("current_date",-7)).limit(1).display()
df_with_week_before.limit(1).display()

#date diff
df_with_week_before.withColumn("diff_between_days",datediff("current_date","week_before")).limit(1).display()

#date format
df_with_week_before.withColumn("week_before",date_format("week_before","dd/MM/yyyy")).limit(1).display()

### Handing null

In [0]:
#droping records having null

df.dropna("all") # drop the record if all column have null value in that record
df.dropna("any").display() #drop every record where any of the column have the null value
df.dropna(subset=["Outlet_Size"]).display() # search for the null in the specified columns

In [0]:
#filling null values

df.fillna("NA").display() # apply to all null
df.fillna({"Item_Weight":-1,"Outlet_size":"NA"}).display() #apply with specific value

df.fillna("NOT", subset=["Outlet_Size"]).display()

### Splitting and Indexing

In [0]:
from pyspark.sql.functions import split,array_contains

df_with_splitted_text=df.withColumn("Outlet_Type",split("Outlet_Type"," ")) # splitting based on a delimiter
df.withColumn("Outlet_Type",split("Outlet_Type"," ")[1]).display() #indexing

#array contains
df_with_splitted_text.withColumn("Type1_flag",array_contains("Outlet_Type","Type1")).display()

### Aggregation Operations

In [0]:
from pyspark.sql.functions import sum,avg,min,max,count, collect_list, collect_set

df.groupBy("Item_Type").agg(count("Item_MRP")).display()

df.groupBy("Item_Type","Outlet_Size").agg(avg("Item_MRP")).alias("avg price by outlet size").display()#group by on two columns

df.groupBy("Item_Type","Outlet_Size").agg(sum("Item_MRP"),avg("Item_MRP")).display() #group by on two column and two agg function

df.groupBy("Item_Type").agg(collect_list("Item_Identifier")).display()
df.groupBy("Item_Type").agg(collect_set("Item_Identifier")).display()

### Pivot

In [0]:
df.groupBy("Item_Type").pivot("Outlet_Size").agg(avg("Item_MRP")).display()

### When Otherwise

In [0]:
from pyspark.sql.functions import when

df_with_veg_column=df.withColumn("isVeg",when(col("Item_Type")=="Meat",False).otherwise(True))
df_with_veg_column.select("Item_Type","isVeg").limit(5).display()

df_with_veg_column.withColumn("expensive_flag",when((col("isVeg")==True) & (col("Item_MRP")>=100),"Veg Expensive")\
                            .when((col("isVeg")==True) & (col("Item_MRP")<100),"Veg inExpensive")\
                            .when((col("isVeg")==False) & (col("Item_MRP")>=100),"Non Veg Expensive")\
                            .otherwise("Non-Veg InExpensive")).select("Item_Type","Item_MRP","isVeg","expensive_flag").display()

### Joins

In [0]:
emp_data = [
    (1, "Alice", 101),
    (2, "Bob", 102),
    (3, "Charlie", 101),
    (4, "David", 103),
    (5, "Eve", None)
]
emp_schema = StructType([
    StructField("emp_id", IntegerType(), True),
    StructField("emp_name", StringType(), True),
    StructField("dept_id", IntegerType(), True)
])
df_emp = spark.createDataFrame(emp_data, schema=emp_schema)
df_emp.display()

dept_data = [
    (101, "Sales", "New York"),
    (102, "Marketing", "London"),
    (103, "IT", "Paris"),
    (104, "HR", "Remote")
]
dept_schema = StructType([
    StructField("dept_id", IntegerType(), True),
    StructField("dept_name", StringType(), True),
    StructField("location", StringType(), True)
])
df_dept = spark.createDataFrame(dept_data, schema=dept_schema)
df_dept.display()

In [0]:
#inner
df_emp.join(df_dept,df_emp["dept_id"]==df_dept["dept_id"],"inner").display()

#left
df_emp.join(df_dept,df_emp["dept_id"]==df_dept["dept_id"],"left").display()

#right
df_emp.join(df_dept,df_emp["dept_id"]==df_dept["dept_id"],"right").display()

#full
df_emp.join(df_dept,df_emp["dept_id"]==df_dept["dept_id"],"full").display()

#anti
df_emp.join(df_dept,df_emp["dept_id"]==df_dept["dept_id"],"anti").display()

### Window functions

In [0]:
from pyspark.sql.window import *
from pyspark.sql.functions import row_number, rank, dense_rank

df.withColumn("row_number",row_number().over(Window.orderBy("Item_Identifier")))\
    .withColumn("rank",rank().over(Window.orderBy("Item_Identifier")))\
        .withColumn("dense_rank",dense_rank().over(Window.orderBy("Item_Identifier"))).limit(5).display()

In [0]:
from pyspark.sql.functions import *

#Running total

df.withColumn("cummulative_sum",sum("Item_MRP").over(Window.orderBy("Item_Type").partitionBy("Item_Type").rowsBetween(Window.unboundedPreceding,Window.currentRow))).limit(5).display()

### User defined functions

In [0]:
def my_func(x):
    if x is None:
        return "NA"
    if x > 10:
        return "Heavy"
    return "Light"

my_udf=udf(my_func)

df.withColumn("weight_flag",my_udf("Item_Weight")).limit(5).display()

### Writing data

In [0]:
#modes : append, overwrite, error, ignore

df.write.format("csv").mode("ignore").save("/FileStore/tables/csv/my-file.csv")

In [0]:
dbutils.fs.ls("/FileStore/tables/csv/")

### Spark SQL

In [0]:
#creating temp view from df
df.createTempView("view")

In [0]:
%sql
select * from view where Item_Weight >10

In [0]:
# writing sql and convert back into df
df_sql=spark.sql("select * from view")
df_sql.limit(5).display()